# Heating Relay Patterns Analysis

This notebook analyzes heating relay on/off patterns across different rooms in the Loxone smart home system.

## Analysis Goals
- Analyze relay switching patterns (on/off cycles) by room
- Calculate heating duty cycles and energy consumption 
- Identify room-specific heating schedules and patterns
- Optimize relay switching for energy efficiency
- Calculate actual power consumption from relay states using room power ratings

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from influxdb_client import InfluxDBClient
from datetime import datetime, timedelta
import pytz
from scipy import signal
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 2. Database Connection

In [ ]:
# InfluxDB connection parameters
INFLUX_URL = "http://192.168.0.201:8086"
INFLUX_TOKEN = "7HrEuj8kzOS1f-0mjU_GT4hS_9gHfdjUT6j5QAM22oDg0z44DsxmiveTGMqTa0Zl1QezDh132utLbXi-IL8h9A=="
INFLUX_ORG = "loxone"
INFLUX_BUCKET = "loxone"

# Room heating power ratings (kW) from your Loxone system
ROOM_POWER = {
    "hosti": 2.02, "chodba_dole": 1.8, "chodba_nahore": 1.2, "koupelna_dole": 0.47, 
    "koupelna_nahore": 0.62, "loznice": 1.2, "obyvak": 4.8, "pokoj_1": 1.2, 
    "pokoj_2": 1.2, "pracovna": 0.82, "satna_dole": 0.82, "satna_nahore": 0.56,
    "spajz": 0.46, "technicka_mistnost": 0.82, "zadveri": 0.82, "zachod": 0.22
}

# Initialize InfluxDB client
client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
query_api = client.query_api()

## 3. Data Loading Functions

In [ ]:
def load_temperature_data(start_date, end_date):
    """Load room temperature data from InfluxDB"""
    query = f'''
    from(bucket: "{INFLUX_BUCKET}")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r["_measurement"] == "temperature")
        |> aggregateWindow(every: 5m, fn: mean, createEmpty: false)
        |> keep(columns: ["_time", "_value", "_field"])
    '''
    result = query_api.query_data_frame(query)
    return result

def load_relay_data(start_date, end_date):
    """Load heating relay on/off data from InfluxDB"""
    query = f'''
    from(bucket: "{INFLUX_BUCKET}")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r._measurement == "relay" and r.tag1 == "heating" and r.room != "kuchyne")
        |> aggregateWindow(every: 15m, fn: mean, createEmpty: false)
        |> keep(columns: ["_time", "_value", "room"])
    '''
    result = query_api.query_data_frame(query)
    return result

def calculate_power_consumption(relay_data):
    """Calculate actual power consumption from relay states and room power ratings"""
    if relay_data.empty:
        return relay_data
    
    # Create power consumption column
    relay_data['power_kw'] = relay_data.apply(
        lambda row: row['_value'] * ROOM_POWER.get(row['room'], 0) if 'room' in row else 0, 
        axis=1
    )
    relay_data['power_w'] = relay_data['power_kw'] * 1000
    
    return relay_data

def analyze_relay_cycles(relay_data):
    """Analyze relay on/off cycling patterns"""
    cycles = {}
    
    for room in relay_data['room'].unique():
        room_data = relay_data[relay_data['room'] == room].copy()
        room_data = room_data.sort_values('_time')
        
        # Calculate duty cycle (percentage of time relay is on)
        duty_cycle = room_data['_value'].mean() * 100
        
        # Count switching events (state changes)
        room_data['state_change'] = room_data['_value'].diff().abs() > 0.1
        switches = room_data['state_change'].sum()
        
        # Calculate average on/off periods
        on_periods = []
        off_periods = []
        current_state = None
        current_start = None
        
        for _, row in room_data.iterrows():
            state = row['_value'] > 0.5  # Consider >0.5 as "on"
            
            if current_state is None:
                current_state = state
                current_start = row['_time']
            elif current_state != state:
                duration = (row['_time'] - current_start).total_seconds() / 60  # minutes
                
                if current_state:  # Was on, now off
                    on_periods.append(duration)
                else:  # Was off, now on
                    off_periods.append(duration)
                
                current_state = state
                current_start = row['_time']
        
        cycles[room] = {
            'duty_cycle_percent': duty_cycle,
            'total_switches': switches,
            'avg_on_minutes': np.mean(on_periods) if on_periods else 0,
            'avg_off_minutes': np.mean(off_periods) if off_periods else 0,
            'total_on_time_hours': sum(on_periods) / 60 if on_periods else 0,
            'power_rating_kw': ROOM_POWER.get(room, 0),
            'total_energy_kwh': (sum(on_periods) / 60) * ROOM_POWER.get(room, 0) if on_periods else 0
        }
    
    return cycles

## 4. Load and Prepare Data

In [ ]:
# Define analysis period - last 30 days for comprehensive relay pattern analysis
end_date = datetime.now(pytz.UTC)
start_date = end_date - timedelta(days=30)

# Load data
print(f"Loading relay and temperature data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
temp_data = load_temperature_data(start_date.isoformat(), end_date.isoformat())
relay_data = load_relay_data(start_date.isoformat(), end_date.isoformat())

print(f"Temperature data shape: {temp_data.shape}")
print(f"Relay data shape: {relay_data.shape}")

if not relay_data.empty:
    # Calculate power consumption from relay states
    relay_data = calculate_power_consumption(relay_data)
    
    # Get list of rooms with heating relays
    rooms = relay_data['room'].unique()
    print(f"\nRooms with heating relays: {list(rooms)}")
    
    # Show relay state statistics
    print(f"\nRelay Statistics:")
    for room in rooms:
        room_relays = relay_data[relay_data['room'] == room]
        on_time_pct = (room_relays['_value'].mean() * 100)
        max_power = ROOM_POWER.get(room, 0) * 1000
        print(f"  {room:20s}: {on_time_pct:5.1f}% on-time, {max_power:6.0f}W max power")
else:
    print("No relay data found - check if heating system was active during this period")

## 5. Room Temperature Analysis

In [ ]:
# Analyze relay switching patterns for each room
if not relay_data.empty:
    # Create relay state timeline plot
    fig = make_subplots(
        rows=len(rooms), cols=1,
        subplot_titles=[f'Heating Relay: {room} ({ROOM_POWER.get(room, 0):.1f}kW)' for room in rooms],
        shared_xaxes=True
    )
    
    for i, room in enumerate(rooms, 1):
        room_data = relay_data[relay_data['room'] == room]
        
        # Create binary state plot (0=off, 1=on)
        fig.add_trace(
            go.Scatter(
                x=room_data['_time'],
                y=room_data['_value'],
                name=f'{room} Relay',
                mode='lines',
                line=dict(width=2, shape='hv'),  # Step-like appearance for relay states
                fill='tonexty' if i > 1 else 'tozeroy',
                fillcolor=f'rgba({50 + i*30}, {100 + i*20}, {200 - i*10}, 0.3)'
            ),
            row=i, col=1
        )
        
        # Add power consumption on secondary y-axis
        fig.add_trace(
            go.Scatter(
                x=room_data['_time'],
                y=room_data['power_w'],
                name=f'{room} Power',
                mode='lines',
                line=dict(color='red', width=1),
                yaxis=f'y{i}' if i > 1 else 'y',
                visible='legendonly'  # Hidden by default
            ),
            row=i, col=1
        )
    
    fig.update_layout(
        height=150*len(rooms),
        title_text="Heating Relay States and Power Consumption",
        showlegend=True
    )
    
    # Update y-axes
    for i in range(1, len(rooms)+1):
        fig.update_yaxes(title_text="Relay State", range=[-0.1, 1.1], row=i, col=1)
    
    fig.update_xaxes(title_text="Time", row=len(rooms), col=1)
    fig.show()
else:
    print("No relay data available for visualization")

## 6. Heating Schedule Analysis

In [ ]:
# Analyze daily and hourly heating patterns
if not relay_data.empty:
    relay_data['hour'] = pd.to_datetime(relay_data['_time']).dt.hour
    relay_data['weekday'] = pd.to_datetime(relay_data['_time']).dt.day_name()
    relay_data['is_weekend'] = pd.to_datetime(relay_data['_time']).dt.weekday >= 5
    
    # Calculate hourly duty cycles
    hourly_patterns = relay_data.groupby(['hour', 'is_weekend', 'room'])['_value'].mean().reset_index()
    hourly_patterns['duty_cycle_percent'] = hourly_patterns['_value'] * 100
    
    # Plot heating patterns by hour and day type
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    plot_rooms = rooms[:4] if len(rooms) >= 4 else rooms
    
    for idx, room in enumerate(plot_rooms):
        room_patterns = hourly_patterns[hourly_patterns['room'] == room]
        weekday_data = room_patterns[~room_patterns['is_weekend']]
        weekend_data = room_patterns[room_patterns['is_weekend']]
        
        ax = axes[idx]
        ax.plot(weekday_data['hour'], weekday_data['duty_cycle_percent'], 'b-', 
                label='Weekday', linewidth=2, marker='o')
        ax.plot(weekend_data['hour'], weekend_data['duty_cycle_percent'], 'r-', 
                label='Weekend', linewidth=2, marker='s')
        ax.set_xlabel('Hour of Day')
        ax.set_ylabel('Heating Duty Cycle (%)')
        ax.set_title(f'{room} - Daily Heating Pattern ({ROOM_POWER.get(room, 0):.1f}kW)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xlim(0, 23)
        ax.set_ylim(0, max(100, room_patterns['duty_cycle_percent'].max() * 1.1))
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\\nHeating Schedule Analysis:")
    print("=" * 50)
    for room in rooms:
        room_data = hourly_patterns[hourly_patterns['room'] == room]
        if not room_data.empty:
            peak_hour = room_data.loc[room_data['duty_cycle_percent'].idxmax(), 'hour']
            peak_duty = room_data['duty_cycle_percent'].max()
            avg_duty = room_data['duty_cycle_percent'].mean()
            print(f"{room:20s}: Peak {peak_duty:5.1f}% at {peak_hour:2.0f}:00, Avg {avg_duty:5.1f}%")
else:
    print("No relay data available for schedule analysis")

## 7. Setpoint vs Actual Temperature Analysis

In [ ]:
# Compare setpoints with actual temperatures
# This section would merge temperature and setpoint data

# Placeholder for analysis
print("Setpoint vs Actual Temperature Analysis")
print("=======================================")
print("This analysis would show:")
print("- How well rooms maintain their setpoint temperatures")
print("- Average deviation from setpoint by room")
print("- Time to reach setpoint after changes")
print("- Overshoot/undershoot patterns")

# Example visualization structure
fig, ax = plt.subplots(figsize=(12, 8))
# Placeholder scatter plot
ax.set_xlabel('Setpoint Temperature (°C)')
ax.set_ylabel('Actual Temperature (°C)')
ax.set_title('Setpoint vs Actual Temperature')
ax.plot([18, 24], [18, 24], 'r--', label='Perfect Match')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 8. Energy Consumption Analysis

In [ ]:
# Analyze total energy consumption from relay data
if not relay_data.empty:
    # Calculate daily total energy consumption across all rooms
    relay_data['date'] = pd.to_datetime(relay_data['_time']).dt.date
    
    # Calculate energy per 15-minute interval: power_kw * (15/60) hours
    relay_data['energy_kwh_interval'] = relay_data['power_kw'] * 0.25  # 15 minutes = 0.25 hours
    
    # Daily consumption by room
    daily_by_room = relay_data.groupby(['date', 'room'])['energy_kwh_interval'].sum().reset_index()
    
    # Total daily consumption
    daily_total = relay_data.groupby('date')['energy_kwh_interval'].sum()
    
    # Hourly consumption pattern
    hourly_consumption = relay_data.groupby('hour')['power_kw'].sum()  # Total power across all rooms
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Daily total consumption
    ax1.bar(range(len(daily_total)), daily_total.values, color='orange', alpha=0.7)
    ax1.set_ylabel('Energy Consumption (kWh)')
    ax1.set_title('Daily Total Heating Energy Consumption')
    ax1.set_xlabel('Days')
    ax1.grid(True, alpha=0.3)
    
    # 2. Hourly power pattern
    ax2.plot(hourly_consumption.index, hourly_consumption.values, 'b-', linewidth=2, marker='o')
    ax2.fill_between(hourly_consumption.index, hourly_consumption.values, alpha=0.3)
    ax2.set_xlabel('Hour of Day')
    ax2.set_ylabel('Total Heating Power (kW)')
    ax2.set_title('Average Hourly Heating Power (All Rooms)')
    ax2.set_xlim(0, 23)
    ax2.grid(True, alpha=0.3)
    
    # 3. Energy consumption by room (stacked)
    room_daily_pivot = daily_by_room.pivot(index='date', columns='room', values='energy_kwh_interval').fillna(0)
    room_daily_pivot.plot(kind='bar', stacked=True, ax=ax3, legend=True)
    ax3.set_ylabel('Energy Consumption (kWh)')
    ax3.set_title('Daily Energy Consumption by Room')
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. Room power ratings comparison
    room_powers = [ROOM_POWER.get(room, 0) for room in rooms]
    ax4.bar(rooms, room_powers, color='skyblue', alpha=0.7)
    ax4.set_ylabel('Heating Power Rating (kW)')
    ax4.set_title('Room Heating Power Ratings')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display statistics
    print(f"\\nRelay-Based Energy Statistics:")
    print("=" * 50)
    print(f"Analysis period: {len(daily_total)} days")
    print(f"Average daily consumption: {daily_total.mean():.2f} kWh")
    print(f"Peak daily consumption: {daily_total.max():.2f} kWh")
    print(f"Total consumption (period): {daily_total.sum():.2f} kWh")
    print(f"Estimated annual consumption: {(daily_total.mean() * 365):.0f} kWh")
    
    # Peak heating times
    peak_hour = hourly_consumption.idxmax()
    peak_power = hourly_consumption.max()
    print(f"Peak heating hour: {peak_hour}:00 ({peak_power:.1f} kW total)")
    
    # Room-specific statistics
    print(f"\\nRoom-Specific Energy Analysis:")
    print("-" * 40)
    for room in rooms:
        room_data = relay_data[relay_data['room'] == room]
        total_energy = room_data['energy_kwh_interval'].sum()
        duty_cycle = room_data['_value'].mean() * 100
        power_rating = ROOM_POWER.get(room, 0)
        print(f"{room:20s}: {total_energy:6.1f} kWh, {duty_cycle:5.1f}% duty, {power_rating:4.1f}kW rated")
else:
    print("No relay data available for energy analysis")

## 9. Room Efficiency Analysis

In [ ]:
# Analyze relay switching efficiency and cycles
if not relay_data.empty:
    # Get detailed cycling analysis
    cycle_analysis = analyze_relay_cycles(relay_data)
    
    # Convert to DataFrame for easier analysis
    cycle_df = pd.DataFrame.from_dict(cycle_analysis, orient='index')
    cycle_df = cycle_df.round(2)
    
    print("Relay Cycling Analysis")
    print("=" * 60)
    print(f"{'Room':<20} {'Duty%':<8} {'Switches':<10} {'Avg On(min)':<12} {'Avg Off(min)':<12} {'Energy(kWh)':<12}")
    print("-" * 60)
    
    for room in cycle_df.index:
        stats = cycle_df.loc[room]
        print(f"{room:<20} {stats['duty_cycle_percent']:<8.1f} {stats['total_switches']:<10.0f} "
              f"{stats['avg_on_minutes']:<12.1f} {stats['avg_off_minutes']:<12.1f} {stats['total_energy_kwh']:<12.1f}")
    
    # Visualize cycling patterns
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Duty cycle comparison
    ax1.bar(cycle_df.index, cycle_df['duty_cycle_percent'], color='lightblue', alpha=0.7)
    ax1.set_ylabel('Duty Cycle (%)')
    ax1.set_title('Heating Duty Cycle by Room')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # 2. Switching frequency
    ax2.bar(cycle_df.index, cycle_df['total_switches'], color='orange', alpha=0.7)
    ax2.set_ylabel('Total Switches')
    ax2.set_title('Relay Switching Frequency by Room')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    # 3. Average on/off times
    x = np.arange(len(cycle_df.index))
    width = 0.35
    ax3.bar(x - width/2, cycle_df['avg_on_minutes'], width, label='Avg On Time', alpha=0.7)
    ax3.bar(x + width/2, cycle_df['avg_off_minutes'], width, label='Avg Off Time', alpha=0.7)
    ax3.set_ylabel('Minutes')
    ax3.set_title('Average On/Off Periods')
    ax3.set_xticks(x)
    ax3.set_xticklabels(cycle_df.index, rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Energy consumption vs power rating
    ax4.scatter(cycle_df['power_rating_kw'], cycle_df['total_energy_kwh'], 
                s=cycle_df['duty_cycle_percent']*5, alpha=0.6, c=cycle_df['duty_cycle_percent'], cmap='viridis')
    ax4.set_xlabel('Power Rating (kW)')
    ax4.set_ylabel('Total Energy Consumption (kWh)')
    ax4.set_title('Energy vs Power Rating (size = duty cycle)')
    
    # Add room labels
    for room in cycle_df.index:
        ax4.annotate(room, (cycle_df.loc[room, 'power_rating_kw'], cycle_df.loc[room, 'total_energy_kwh']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Efficiency analysis
    print(f"\\nRelay Efficiency Analysis:")
    print("-" * 40)
    
    # Identify rooms with frequent switching (potentially inefficient)
    avg_switches = cycle_df['total_switches'].mean()
    frequent_switchers = cycle_df[cycle_df['total_switches'] > avg_switches * 1.5]
    
    if not frequent_switchers.empty:
        print(f"Rooms with high switching frequency (may need tuning):")
        for room in frequent_switchers.index:
            switches = frequent_switchers.loc[room, 'total_switches']
            print(f"  {room}: {switches:.0f} switches (avg: {avg_switches:.0f})")
    
    # Identify rooms with very short cycles (potentially inefficient)
    short_cycle_rooms = cycle_df[cycle_df['avg_on_minutes'] < 10]
    if not short_cycle_rooms.empty:
        print(f"\\nRooms with short heating cycles (<10 min):")
        for room in short_cycle_rooms.index:
            on_time = short_cycle_rooms.loc[room, 'avg_on_minutes']
            print(f"  {room}: {on_time:.1f} min average on-time")
    
    # Calculate overall system efficiency
    total_energy = cycle_df['total_energy_kwh'].sum()
    total_capacity = sum(ROOM_POWER.values())
    analysis_hours = (end_date - start_date).total_seconds() / 3600
    theoretical_max = total_capacity * analysis_hours
    system_efficiency = (total_energy / theoretical_max) * 100 if theoretical_max > 0 else 0
    
    print(f"\\nSystem Overview:")
    print(f"Total heating capacity: {total_capacity:.1f} kW")
    print(f"Analysis period: {analysis_hours:.0f} hours")
    print(f"Total energy consumed: {total_energy:.1f} kWh")
    print(f"System utilization: {system_efficiency:.1f}% of theoretical maximum")
    
else:
    print("No relay data available for cycling analysis")

## 10. Optimization Opportunities

In [ ]:
# Identify optimization opportunities
print("Heating Optimization Opportunities")
print("=================================")

# 1. Identify overheated periods
if not temp_data.empty:
    overheated = temp_data[temp_data['_value'] > 23]  # Assuming 23°C as upper comfort limit
    if not overheated.empty:
        overheated_hours = overheated.groupby('hour').size()
        print(f"\n1. Overheating detected during hours: {list(overheated_hours.index)}")
    
    # 2. Identify rooms with high variability
    if 'room' in temp_data.columns:
        high_variability = room_stats[room_stats['std'] > 1.0]
        if not high_variability.empty:
            print(f"\n2. Rooms with high temperature variability: {list(high_variability.index)}")
    
    # 3. Identify potential schedule optimizations
    night_temps = temp_data[temp_data['hour'].between(0, 6)]
    avg_night_temp = night_temps.groupby('room')['_value'].mean()
    high_night_temp = avg_night_temp[avg_night_temp > 20]
    if not high_night_temp.empty:
        print(f"\n3. Rooms with high night temperatures: {list(high_night_temp.index)}")
        print("   Consider reducing night setpoints for energy savings")

# 4. Calculate potential savings
print("\n4. Potential Energy Savings:")
print("   - 1°C reduction in average temperature: ~6% energy savings")
print("   - Optimized scheduling: 10-15% potential savings")
print("   - Zone-based control: 5-10% additional savings")

## 11. Key Findings and Recommendations for Relay-Based Heating

### Summary of Relay Analysis

Based on the heating relay pattern analysis, the key findings are:

1. **Relay Switching Patterns**
   - Binary on/off states control individual room heating
   - Each room has specific power rating (0.22kW to 4.8kW)
   - Duty cycles indicate heating demand by room and time
   - Switching frequency shows system responsiveness

2. **Energy Consumption Calculation**
   - Energy = Relay State (0/1) × Power Rating × Time
   - 15-minute intervals provide good resolution
   - Total system capacity: ~18.6kW across all rooms
   - Actual consumption based on relay duty cycles

3. **Optimization Opportunities from Relay Data**
   - High-frequency switching may indicate poor tuning
   - Short on-cycles suggest oversized heating or aggressive control
   - Low duty cycles in high-power rooms indicate efficiency
   - Scheduling patterns reveal occupancy and comfort preferences

### Relay-Specific Recommendations

1. **Immediate Actions**
   - Monitor rooms with >100 switches/day for control tuning
   - Adjust deadband settings for rooms with <10 min cycles
   - Schedule analysis shows optimal heating windows
   - Consider zone grouping for similar usage patterns

2. **Medium-term Improvements**
   - Implement predictive relay control based on occupancy
   - Optimize switching algorithms to reduce wear
   - Use outdoor temperature compensation for relay timing
   - Group low-power rooms for coordinated operation

3. **Long-term Strategies**
   - Upgrade to proportional valve control where beneficial
   - Implement smart relay scheduling based on patterns
   - Consider thermal mass in relay switching logic
   - Integrate with renewable energy for optimal timing

### Relay System Performance Metrics

- **Duty Cycle Efficiency**: Lower cycles with maintained comfort
- **Switching Optimization**: Reduce unnecessary state changes  
- **Power Factor**: Balance high/low power rooms effectively
- **Response Time**: Minimize time to reach target temperatures
- **Energy Distribution**: Optimize power allocation across zones